In [9]:
# 合并dataframe
##############################
# 合并分时数据
##############################
import pickle
import efinance as ef
import pandas as pd
import numpy as np


today_df = ef.stock.get_realtime_quotes()
today_df = today_df.drop(['量比', '动态市盈率','总市值','流通市值','行情ID','市场类型','更新时间','昨日收盘'], axis=1)
today_df = today_df.rename(columns={'最新交易日': '日期'})
today_df = today_df.rename(columns={'今开': '开盘'})
today_df = today_df.rename(columns={'最新价': '收盘'})
today_df['日期'] = pd.to_datetime(today_df['日期'], format='%Y-%m-%d')
today_df.replace('-', float('nan'), inplace=True)
today_df['开盘'] = today_df['开盘'].astype(float)
today_df['最高'] = today_df['最高'].astype(float)
today_df['最低'] = today_df['最低'].astype(float)
today_df['收盘'] = today_df['收盘'].astype(float)
today_df['成交量'] = today_df['成交量'].astype(float)
today_df['成交额'] = today_df['成交额'].astype(float)
today_df['涨跌幅'] = today_df['涨跌幅'].astype(float)
today_df['涨跌额'] = today_df['涨跌额'].astype(float)
today_df['换手率'] = today_df['换手率'].astype(float)

with open('new_total_df.pkl', 'rb') as file:
    total_df = pickle.load(file)
    
new_total_df = pd.concat([total_df,today_df],axis=0)


def get_log_growth(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        df['收盘'] = pd.to_numeric(df['收盘'], errors='coerce')

        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期收盘'] = df.groupby('股票代码')['收盘'].shift(1)

        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['收盘涨幅对数'] = np.log(df['收盘'] / df['滞后一期收盘'])
        return df
    
def get_volume_change(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        df['成交量'] = pd.to_numeric(df['成交量'], errors='coerce')
        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期成交量'] = df.groupby('股票代码')['成交量'].shift(1)
        df['滞后一期成交量'] = df['滞后一期成交量'].replace(0, np.nan)
        # Calculate the percentage change in trade volume
        df['成交量变化百分比'] = (df['成交量'] - df['滞后一期成交量']) / df['滞后一期成交量'] * 100
        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['成交量涨幅对数'] = np.log(df['成交量'] / df['滞后一期成交量'])
        return df
    
new_total_df = get_log_growth(new_total_df)
new_total_df = get_volume_change(new_total_df)
new_total_df



,股票名称,股票代码,日期,开盘,收盘,最高,最低,成交量,成交额,涨跌幅,涨跌额,换手率,滞后一期收盘,收盘涨幅对数,滞后一期成交量,成交量变化百分比,成交量涨幅对数
7458,平安银行,000001,2022-07-04,14.44,14.43,14.47,14.24,834968.0,1.241453e+09,0.14,0.02,0.43,NaN,NaN,NaN,NaN,NaN
7459,平安银行,000001,2022-07-05,14.48,14.40,14.76,14.31,1115172.0,1.674719e+09,-0.21,-0.03,0.57,14.43,-0.002081,834968.0,33.558651,0.289371
7460,平安银行,000001,2022-07-06,14.32,14.02,14.42,13.95,1043335.0,1.520023e+09,-2.64,-0.38,0.54,14.40,-0.026743,1115172.0,-6.441787,-0.066586
7461,平安银行,000001,2022-07-07,14.03,13.91,14.14,13.87,730161.0,1.057697e+09,-0.78,-0.11,0.38,14.02,-0.007877,1043335.0,-30.016629,-0.356913
7462,平安银行,000001,2022-07-08,13.99,14.03,14.14,13.86,639698.0,9.288504e+08,0.86,0.12,0.33,13.91,0.008590,730161.0,-12.389459,-0.132269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,美心翼申,873833,2023-11-28,14.66,17.00,20.28,14.65,78901.0,1.350266e+08,7.59,1.20,29.76,15.80,0.073203,71012.0,11.109390,0.105345
5433,美心翼申,873833,2023-11-29,16.88,15.28,17.78,14.50,58721.0,9.403090e+07,-10.12,-1.72,22.15,17.00,-0.106669,78901.0,-25.576355,-0.295396
0,美心翼申,873833,2023-11-30,14.69,14.25,15.67,14.13,30554.0,4.482326e+07,-6.74,-1.03,11.53,15.28,-0.069788,58721.0,-47.967507,-0.653302
0,美心翼申,873833,2023-12-01,14.23,13.18,14.62,13.15,27368.0,3.743917e+07,-7.51,-1.07,10.32,14.25,-0.078056,30554.0,-10.427440,-0.110121


In [4]:
###### Saving 收盘后再跑 ##########

new_total_df2 = new_total_df.drop(['滞后一期收盘','收盘涨幅对数','滞后一期成交量','成交量变化百分比','成交量涨幅对数'],axis = 1)
# new_total_df = get_log_growth(new_total_df)
# new_total_df = get_volume_change(new_total_df)
# new_total_df.replace('-', float('nan'), inplace=True)
# new_total_df['开盘'] = new_total_df['开盘'].astype(float)
# new_total_df['最高'] = new_total_df['最高'].astype(float)
# new_total_df['最低'] = new_total_df['最低'].astype(float)
# new_total_df['收盘'] = new_total_df['收盘'].astype(float)
# new_total_df['成交量'] = new_total_df['成交量'].astype(float)
# new_total_df['成交额'] = new_total_df['成交额'].astype(float)
# new_total_df['涨跌幅'] = new_total_df['涨跌幅'].astype(float)
# new_total_df['涨跌额'] = new_total_df['涨跌额'].astype(float)
# new_total_df['换手率'] = new_total_df['换手率'].astype(float)
new_total_df2
# Assuming you have a dictionary called 'my_dict'
with open('new_total_df1.pkl', 'wb') as file:
    pickle.dump(new_total_df2, file)

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import pickle



    
def get_log_growth(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)

        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期收盘'] = df.groupby('股票代码')['收盘'].shift(1)

        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['收盘涨幅对数'] = np.log(df['收盘'] / df['滞后一期收盘'])
        return df


def get_volume_change(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期成交量'] = df.groupby('股票代码')['成交量'].shift(1)
        df['滞后一期成交量'] = df['滞后一期成交量'].replace(0, np.nan)
        # Calculate the percentage change in trade volume
        df['成交量变化百分比'] = (df['成交量'] - df['滞后一期成交量']) / df['滞后一期成交量'] * 100
        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['成交量涨幅对数'] = np.log(df['成交量'] / df['滞后一期成交量'])
        return df
# Assuming you have the start and end tactic dates as strings
start_tactic_date = '20220101'
end_tactic_date =  '20241231'
# Convert the tactic dates to datetime objects
start_date = pd.to_datetime(start_tactic_date, format='%Y%m%d')
end_date = pd.to_datetime(end_tactic_date, format='%Y%m%d')
# Generate the series of working days
working_days_total = pd.bdate_range(start=start_date, end=end_date)
national_holiday_list = ['2023-09-29','2023-10-02','2023-10-03','2023-10-04','2023-10-05','2023-10-06']
May_first = ['2023-05-01','2023-05-02','2023-05-03']
Qingming = ['2023-04-05']
Spring_festival = ['2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27']

all_holiday = national_holiday_list+May_first+Qingming+Spring_festival

for day in all_holiday:
    if day in working_days_total:
        working_days_total = working_days_total.drop(day)
        
def get_filtered_df(df, set_date,interval):
    set_date = pd.to_datetime(set_date, format='%Y-%m-%d')
    index = working_days_total.get_loc(set_date)
    # Get the date that is 30 business days before set_date
    start_date = working_days_total[index-(interval-1)]
        #set_date - BDay(interval-1) 
    filtered_df = df[(df['日期'] >= start_date) & (df['日期'] <= set_date)]
    return filtered_df

# def get_filtered_df(df, set_date, interval):
#     # Convert set_date to the desired format
# #     set_date = pd.to_datetime(set_date, format='%Y%m%d')
#     set_date = pd.to_datetime(set_date, format='%Y%m%d')
#     # Get the date that is 30 business days before set_date
#     start_date = set_date - BDay(interval-1) 
#     filtered_df = df[(df['日期'] >= start_date) & (df['日期'] <= set_date)]
#     return filtered_df


# def get_filtered_df_after(df, set_date, interval):
#     # Convert set_date to the desired format
# #     set_date = pd.to_datetime(set_date, format='%Y%m%d')
#     set_date = pd.to_datetime(set_date, format='%Y-%m-%d')
#     # Get the date that is 30 business days before set_date
#     end_date = working_days_total[index+(interval)]
#     filtered_df = df[(df['日期'] >= set_date) & (df['日期'] <= end_date)]
#     return filtered_df

def get_today(df, set_date):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    filtered_df = df[df['日期'] == set_date]
    return filtered_df



def get_growth(filtered_df):
    total_log_growth = filtered_df.groupby('股票代码')['收盘涨幅对数'].sum()
    total_growth = np.exp(total_log_growth) - 1
    return total_growth



def get_vol_growth(filtered_df):
    total_log_growth = filtered_df.groupby('股票代码')['成交量涨幅对数'].sum()
    total_growth = np.exp(total_log_growth) - 1
    return total_growth


def get_average_vol(filtered_df):
    average_vol = filtered_df.groupby('股票代码')['成交量'].mean()
    return average_vol

# 判断近期连续上涨
def whether_price_rise_in_last_n_days(filtered_df, lowwer, upper):
    # Group by 'Scode' and check if all differences are positive
    is_increasing = filtered_df.groupby('股票代码')['涨跌幅'].apply(lambda x: ((x >= lowwer) & (x < upper)).all())
    return is_increasing



# 判断近期成交量连续上涨
def whether_trade_vol_rise_in_last_n_days(filtered_df,extent):
    # Check if all percentage changes are within the range of 100% to 120%
    is_within_range = filtered_df.groupby('股票代码')['成交量变化百分比'].apply(lambda x: (x >= extent).all())
    return is_within_range

# 判断换手率大于n%
def whether_turnover_rate_larger_than(filtered_df,bottom):
    # Check if all percentage changes are within the range of 100% to 120%
    is_larger = filtered_df.groupby('股票代码')['换手率'].apply(lambda x: (x >= bottom).all())
    return is_larger

def get_n_days_latter(df,set_date,n_days):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y-%m-%d')
    index = working_days.get_loc(set_date)
    # Get the date that is 30 business days before set_date
    future_day = working_days[index+n_days]
    filtered_df = df[df['日期'] == future_day]
    return filtered_df

    
# last_30_day_growth_less_than_20p, total rise in a period
def period_price_condition(last_period, R_l, R_h):
    last_period_growth = get_growth(last_period)
    last_period_growth_condition = last_period_growth[(last_period_growth >= R_l) & (last_period_growth < R_h)]
    last_period_growth_condition = list(last_period_growth_condition.index)
    set1 = set(last_period_growth_condition)
    return set1

# period_vol_condition
def period_vol_condition(last_period, R_l, R_h):
    last_period_growth = get_vol_growth(last_period)
    last_period_growth_condition = last_period_growth[(last_period_growth >= R_l) & (last_period_growth < R_h)]
    set1 = set(last_period_growth_condition.index)
    return set1

# price_rise_in_last_3_days, rise continiously
def recent_price_condition(new2_df,set_date,ndays,nday_lw,nday_up):
    # price rice in ndays1
    last_n_days = get_filtered_df(new2_df, set_date, ndays)
    price_rise_ndays = whether_price_rise_in_last_n_days(last_n_days,lowwer=nday_lw, upper=nday_up)
    price_rise_ndays = price_rise_ndays.loc[price_rise_ndays]
    price_rise_ndays = list(price_rise_ndays.index) # is a list
    set2 = set(price_rise_ndays)
    return set2


# volume_twice_times_of_montly_avg
def today_vol_condition(today, period, times):
    today_vol = get_average_vol(today)
    last_period_avg_vol = get_average_vol(period)
    vol_series = pd.concat([today_vol, last_period_avg_vol],axis=1).set_axis(['Series1', 'Series2'], axis=1)
    volume_times_of_period_avg = vol_series[vol_series['Series1'] > times*vol_series['Series2']]
    volume_times_of_period_avg = list(volume_times_of_period_avg.index)
    set3 = set(volume_times_of_period_avg)
    return set3

# price_rise_in_last_1_days
def today_price_condition(today,onelw,oneup):
    # price rice today
    price_rise_today = whether_price_rise_in_last_n_days(today,lowwer=onelw, upper=oneup)
    price_rise_today = price_rise_today.loc[price_rise_today]
    price_rise_today = list(price_rise_today.index) # is a list
    set4 = set(price_rise_today)
    return set4


def vol_rise_in_last_n_days(new2_df, set_date,ndays,extent):
    last_n_days = get_filtered_df(new2_df, set_date, ndays)
    vol_rise_in_last_n_days = whether_trade_vol_rise_in_last_n_days(last_n_days,extent=extent)
    vol_rise_in_last_n_days = vol_rise_in_last_n_days.loc[vol_rise_in_last_n_days]
    vol_rise_in_last_n_days = list(vol_rise_in_last_n_days.index) # is a list
    set5 = set(vol_rise_in_last_n_days)
    return set5
    

def today_turnover_condition(today, turnoverRate):
    turnover_larger_than = whether_turnover_rate_larger_than(today,turnoverRate)
    turnover_larger_than = turnover_larger_than[turnover_larger_than]
    set6 = set(turnover_larger_than.index)
    return set6



def get_portfolio_return(df,set_date,scode_list,n_days):
    # buy at close price
    today = get_today(df, set_date)
    selected_scode = today[today['股票代码'].isin(scode_list)]
    buy_at = 100*selected_scode['收盘'].sum()
    # sell n days latter
    future_n_days = get_n_days_latter(df,set_date,n_days)
    selected_scode = future_n_days[future_n_days['股票代码'].isin(scode_list)]
    sell_at = 100*selected_scode['收盘'].sum()
    return (sell_at/buy_at-1)

In [12]:
######################### 当日策略开始 #####################


import time

# set_date = '20230926'
# day = pd.to_datetime(set_date, format='%Y%m%d')
    

import pandas as pd

# 请在end_tactic_date修改至今天的日期
start_tactic_date = '20231110' #str(input('输入测试开始时间：（例：20231001）'))
end_tactic_date =  '20231204' #str(input('输入测试结束时间：（例：20231031）')







# Convert the tactic dates to datetime objects
start_date = pd.to_datetime(start_tactic_date, format='%Y%m%d')
end_date = pd.to_datetime(end_tactic_date, format='%Y%m%d')
# Generate the series of working days
working_days = pd.bdate_range(start=start_date, end=end_date)
national_holiday_list = ['2023-09-29','2023-10-02','2023-10-03','2023-10-04','2023-10-05','2023-10-06']
May_first = ['2023-05-01','2023-05-02','2023-05-03']
Qingming = ['2023-04-05']
Spring_festival = ['2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27']

all_holiday = national_holiday_list+May_first+Qingming+Spring_festival

for day in all_holiday:
    if day in working_days:
        working_days = working_days.drop(day)

  
# 涨幅限制用于切片，涨幅天数是最大限制。 

start_time_original = time.time()
# portfolio args
return_rate_total = []
day_list = []
code_list = []


nday_sold = 1 #int(input("请设置在购买后第几天卖出："))
prek, prek_last, mtimes =3, 10, 2# map(int,input("V0为D-t之前的n天的均值的m倍以上，请填写t、n和m，用空格隔开：（例：2 10 2，表示D-2之前10天，2倍）").split())
V1_l, V1_h = 1.2, 3 #map(float,input("今日/昨日成交量倍数区间，VO/V-1，范围(X1Y1)，给出X1与Y1并用空格隔开：（例：1.2 3）").split())
V2_l, V2_h = 0, 1000 #map(float,input("今日/前天成交量倍数区间，V0/V-2，范围(X2Y2)，给出X2与Y2并用空格隔开：（例：1.5 4）").split())
V3_l, V3_h = 1, 2 # map(float,input("昨天/前天成交量倍数区间，V-1/V-2，范围(X3Y3)，给出X3与Y3并用空格隔开：（例：1 2）").split())
Mtimes = 2 #int(input("请设置(V0/V-1)/(V-1/V-2) < Mtimes中的 Mtimes"))
turnoverRate0 = 3 #float(input("今日换手率大于，V0>X4%，请给出X4：（例：3，表示3%）"))
R1_h = 1 #float(input("D-3到D-7涨幅低于R1%，请给出R1（例：1，表示1%）"))
R2_h =1 # float(input("D-3涨幅低于R2，请给出R2%（例：1，表示1%）"))
R3_l = 5 #float(input("P0/P3-1高于R3%，请给出R3（例：5，表示5%）"))
rnday, R4_l, R4_h = 3, 0, 8 #map(float,input("最近k天中每天涨幅在(L,H)之间，请给出L和H并用空格隔开：（例：3 0 8，表示在0%到8%之间）").split())
rnday = int(rnday)
select_df = pd.DataFrame(columns=['股票代码','股票名称','日期','成交量','换手率','今日涨幅','近三日涨幅']).set_index('股票代码')


for index, day in enumerate(working_days):
    start_time = time.time()
    slide_df = get_filtered_df(new_total_df, working_days[index], 30)

    print('Selecting stocks at {}'.format(day))
    # 计算今天的情况
    today = get_today(slide_df, day)
    today_vol = get_average_vol(today)
    today_growth = get_growth(today)
    se1, set2, set3, set4, set5, set6, set7, set8, set9, set10, set11 = set(), set(), set(), set(), set(), set(), set(), set(), set(), set(), set()
    # 最近一段时间成交量条件：V0为D-2之前10天成交量的均值的m倍以上
    set_period = get_filtered_df(slide_df, working_days[index-prek], prek_last) #  day - BDay(prek)
    period_vol_avg = get_average_vol(set_period)
    vol_series = pd.concat([today_vol, period_vol_avg],axis=1).set_axis(['Series1', 'Series2'], axis=1)
    volume_m_times_of_period_avg = vol_series[vol_series['Series1'] > mtimes*vol_series['Series2']]
    set1 = set(volume_m_times_of_period_avg.index)
    if set1:
        # 今日/昨日成交量倍数区间，VO/V-1，范围(X1Y1)
        slide_minus1to0 = get_filtered_df(slide_df, working_days[index], 1)
        set2 = period_vol_condition(slide_minus1to0, V1_l-1, V1_h-1)
        set1 = set1.intersection(set2)
        print()
    if set2:
        # 今日/前天成交量倍数区间，V0/V-2，范围(X2Y2)
        slide_minus2to0 = get_filtered_df(slide_df, working_days[index], 2)
        set3 = period_vol_condition(slide_minus2to0, V2_l-1, V2_h-1)   
        set1 = set1.intersection(set3)
    if set3:
        # 昨天/前天成交量倍数区间，V-1/V-2，范围(X3Y3)
        slide_minus2to1 = get_filtered_df(slide_df, working_days[index-1], 1) # day-BDay(1)
        set4 = period_vol_condition(slide_minus2to1, V3_l-1, V3_h-1) 
        set1 = set1.intersection(set4)
    if set4:
        # 今日换手率大于，V0>X4%
        set5 = today_turnover_condition(today, turnoverRate0)
        set1 = set1.intersection(set5)
    if set5:
        # 最近几天涨幅条件1（D-3 to D-7 less than n%)
        slide_minus7to2 = get_filtered_df(slide_df, working_days[index-3], 5) # day - BDay(3)
        set6 = period_price_condition(slide_minus7to2, -10,R1_h/100)
        set1 = set1.intersection(set6)
    if set6:
        # 最近几天涨幅条件2（D-3 less than n%)
        slide_minus3 = get_filtered_df(slide_df, working_days[index-3], 1) # day - BDay(3)
        set7 = period_price_condition(slide_minus3, -10, R2_h/100)
        set1 = set1.intersection(set7)
    if set7:
        # 最近几天涨幅条件3（P0/P-3)-1 more than n%)
        slide_minus0to3 = get_filtered_df(slide_df, working_days[index], 3) #-3 -2 -1 0
        set8 = period_price_condition(slide_minus0to3, R3_l/100, 100)
        set1 = set1.intersection(set8)
    if set8:
        # 最近三天每天涨幅大于0小于8
        set9 = recent_price_condition(slide_df,day,rnday,R4_l,R4_h)
        set1 = set1.intersection(set9)
    if set9:
        slide_minus3p5 = get_filtered_df(slide_df, working_days[index-3], 20)
        set10 = period_price_condition(slide_minus3p5,-100,0.1)
        set1 = set1.intersection(set10)    
        # 
    if set10:
        # (V0/V-1)/(V-1/V-2)<2，倍数Mtimes
        V0_Vm1 = get_vol_growth(slide_minus1to0)
        Vm1_Vm2 = get_vol_growth(slide_minus2to1)
        vol_series = pd.concat([V0_Vm1, Vm1_Vm2],axis=1).set_axis(['V0_Vm1', 'Vm1_Vm2'], axis=1)
        times_of_change = vol_series[(vol_series['V0_Vm1']+1)/(vol_series['Vm1_Vm2']+1) < Mtimes]
        set11 = set(times_of_change.index)
        set1 = set1.intersection(set11)
        
        
    selected_codes = set1
    if selected_codes:
        print('Selected stocks are: {} \n'.format(selected_codes))
        company_list = today[['股票代码','股票名称','日期','成交量','换手率']].set_index('股票代码')
        company_list = company_list.loc[company_list.index.isin(selected_codes)] 
        today_pchange = pd.DataFrame(today_growth).rename(columns={'收盘涨幅对数': '今日涨幅'})
        today_pchange = today_pchange.loc[today_pchange.index.isin(selected_codes)]
        recent3daychange = pd.DataFrame(get_growth(get_filtered_df(slide_df, working_days[index], 3))).rename(columns={'收盘涨幅对数': '近三日涨幅'})
        recent3daychange = recent3daychange.loc[recent3daychange.index.isin(selected_codes)]
        result_series = pd.concat([company_list,today_pchange,recent3daychange],axis=1)
        select_df = pd.concat([select_df,result_series],axis=0)
    else:
        print('No stock selected.\n')
        
end_time_all = time.time()
execution_time_all = end_time_all-start_time_original

print('Finish execution, using {} seconds'.format(execution_time_all))
select_df['今日涨幅'] = select_df['今日涨幅'].astype(float)
select_df['近三日涨幅'] = select_df['近三日涨幅'].astype(float)
select_df['今日涨幅'] = select_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
select_df['近三日涨幅'] = select_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
select_df['日期'] = pd.to_datetime(select_df['日期'], format='%Y-%m-%d')
select_df.to_excel('select_df.xlsx')

Selecting stocks at 2023-11-10 00:00:00
No stock selected.

Selecting stocks at 2023-11-13 00:00:00
No stock selected.

Selecting stocks at 2023-11-14 00:00:00
No stock selected.

Selecting stocks at 2023-11-15 00:00:00

Selected stocks are: {'301287', '000547', '300988', '600213'} 

Selecting stocks at 2023-11-16 00:00:00

Selected stocks are: {'300698', '300975'} 

Selecting stocks at 2023-11-17 00:00:00

Selected stocks are: {'001317', '600109', '000592', '601801', '002971', '002198', '002453'} 

Selecting stocks at 2023-11-20 00:00:00

Selected stocks are: {'688502', '600237', '300295'} 

Selecting stocks at 2023-11-21 00:00:00

Selected stocks are: {'605179', '873152', '300806', '603716'} 

Selecting stocks at 2023-11-22 00:00:00

Selected stocks are: {'001234', '002086'} 

Selecting stocks at 2023-11-23 00:00:00

Selected stocks are: {'000926', '600708', '603569'} 

Selecting stocks at 2023-11-24 00:00:00

Selected stocks are: {'301211'} 

Selecting stocks at 2023-11-27 00:00:00


<ipython-input-12-5bf0a993615f>:149: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  select_df.to_excel('select_df.xlsx')


In [21]:
######################   新的回测 20231121 ######################


import time

# set_date = '20230926'
# day = pd.to_datetime(set_date, format='%Y%m%d')
    

import pandas as pd

# 请修改测试时间
start_tactic_date = '20221201' #str(input('输入测试开始时间：（例：20231001）'))
end_tactic_date =  '20231121' #str(input('输入测试结束时间：（例：20231031）'))





# Convert the tactic dates to datetime objects
start_date = pd.to_datetime(start_tactic_date, format='%Y%m%d')
end_date = pd.to_datetime(end_tactic_date, format='%Y%m%d')
# Generate the series of working days
working_days = pd.bdate_range(start=start_date, end=end_date)
national_holiday_list = ['2023-09-29','2023-10-02','2023-10-03','2023-10-04','2023-10-05','2023-10-06']
May_first = ['2023-05-01','2023-05-02','2023-05-03']
Qingming = ['2023-04-05']
Spring_festival = ['2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27']

all_holiday = national_holiday_list+May_first+Qingming+Spring_festival

for day in all_holiday:
    if day in working_days:
        working_days = working_days.drop(day)

  
# 涨幅限制用于切片，涨幅天数是最大限制。 

start_time_original = time.time()
# portfolio args
return_rate_total = []
day_list = []
code_list = []


nday_sold = 1 #int(input("请设置在购买后第几天卖出："))
prek, prek_last, mtimes =3, 10, 2# map(int,input("V0为D-t之前的n天的均值的m倍以上，请填写t、n和m，用空格隔开：（例：2 10 2，表示D-2之前10天，2倍）").split())
V1_l, V1_h = 1.2, 3 #map(float,input("今日/昨日成交量倍数区间，VO/V-1，范围(X1Y1)，给出X1与Y1并用空格隔开：（例：1.2 3）").split())
V2_l, V2_h = 0, 1000 #map(float,input("今日/前天成交量倍数区间，V0/V-2，范围(X2Y2)，给出X2与Y2并用空格隔开：（例：1.5 4）").split())
V3_l, V3_h = 1, 2 # map(float,input("昨天/前天成交量倍数区间，V-1/V-2，范围(X3Y3)，给出X3与Y3并用空格隔开：（例：1 2）").split())
Mtimes = 2 #int(input("请设置(V0/V-1)/(V-1/V-2) < Mtimes中的 Mtimes"))
turnoverRate0 = 3 #float(input("今日换手率大于，V0>X4%，请给出X4：（例：3，表示3%）"))
R1_h = 1 #float(input("D-3到D-7涨幅低于R1%，请给出R1（例：1，表示1%）"))
R2_h =1 # float(input("D-3涨幅低于R2，请给出R2%（例：1，表示1%）"))
R3_l = 5 #float(input("P0/P3-1高于R3%，请给出R3（例：5，表示5%）"))
rnday, R4_l, R4_h = 3, 0, 8 #map(float,input("最近k天中每天涨幅在(L,H)之间，请给出L和H并用空格隔开：（例：3 0 8，表示在0%到8%之间）").split())
rnday = int(rnday)
result_df = pd.DataFrame(columns=['股票代码','股票名称','日期','成交量','今日涨幅','近三日涨幅','D+1涨幅','D+2涨幅','D+3涨幅','D+4涨幅','D+5涨幅','D+6涨幅','D+7涨幅','D+8涨幅','D+9涨幅','D+10涨幅','D+11涨幅','D+12涨幅','D+13涨幅','D+14涨幅','D+15涨幅','D+16涨幅','D+17涨幅','D+18涨幅','D+19涨幅','D+20涨幅']).set_index('股票代码')



for index, day in enumerate(working_days):
    start_time = time.time()
    slide_df = get_filtered_df(new_total_df, working_days[index], 30)
    print('Selecting stocks at {}'.format(day))
    # 计算今天的情况
    today = get_today(slide_df, day)
    today_vol = get_average_vol(today)
    today_growth = get_growth(today)
    se1, set2, set3, set4, set5, set6, set7, set8, set9, set10, set11 = set(), set(), set(), set(), set(), set(), set(), set(), set(), set(), set()
    # 最近一段时间成交量条件：V0为D-2之前10天成交量的均值的m倍以上
    set_period = get_filtered_df(slide_df, working_days[index-prek], prek_last) #  day - BDay(prek)
    period_vol_avg = get_average_vol(set_period)
    vol_series = pd.concat([today_vol, period_vol_avg],axis=1).set_axis(['Series1', 'Series2'], axis=1)
    volume_m_times_of_period_avg = vol_series[vol_series['Series1'] > mtimes*vol_series['Series2']]
    set1 = set(volume_m_times_of_period_avg.index)
    if set1:
        # 今日/昨日成交量倍数区间，VO/V-1，范围(X1Y1)
        slide_minus1to0 = get_filtered_df(slide_df, working_days[index], 1)
        set2 = period_vol_condition(slide_minus1to0, V1_l-1, V1_h-1)
        set1 = set1.intersection(set2)
        print()
    if set2:
        # 今日/前天成交量倍数区间，V0/V-2，范围(X2Y2)
        slide_minus2to0 = get_filtered_df(slide_df, working_days[index], 2)
        set3 = period_vol_condition(slide_minus2to0, V2_l-1, V2_h-1)   
        set1 = set1.intersection(set3)
    if set3:
        # 昨天/前天成交量倍数区间，V-1/V-2，范围(X3Y3)
        slide_minus2to1 = get_filtered_df(slide_df, working_days[index-1], 1) # day-BDay(1)
        set4 = period_vol_condition(slide_minus2to1, V3_l-1, V3_h-1) 
        set1 = set1.intersection(set4)
    if set4:
        # 今日换手率大于，V0>X4%
        set5 = today_turnover_condition(today, turnoverRate0)
        set1 = set1.intersection(set5)
    if set5:
        # 最近几天涨幅条件1（D-3 to D-7 less than n%)
        slide_minus7to2 = get_filtered_df(slide_df, working_days[index-3], 5) # day - BDay(3)
        set6 = period_price_condition(slide_minus7to2, -10,R1_h/100)
        set1 = set1.intersection(set6)
    if set6:
        # 最近几天涨幅条件2（D-3 less than n%)
        slide_minus3 = get_filtered_df(slide_df, working_days[index-3], 1) # day - BDay(3)
        set7 = period_price_condition(slide_minus3, -10, R2_h/100)
        set1 = set1.intersection(set7)
    if set7:
        # 最近几天涨幅条件3（P0/P-3)-1 more than n%)
        slide_minus0to3 = get_filtered_df(slide_df, working_days[index], 3) #-3 -2 -1 0
        set8 = period_price_condition(slide_minus0to3, R3_l/100, 100)
        set1 = set1.intersection(set8)
    if set8:
        # 最近三天每天涨幅大于0小于8
        set9 = recent_price_condition(slide_df,day,rnday,R4_l,R4_h)
        set1 = set1.intersection(set9)
    if set9:
        slide_minus3p5 = get_filtered_df(slide_df, working_days[index-3], 20)
        set10 = period_price_condition(slide_minus3p5,-100,0.1)
        set1 = set1.intersection(set10)    
        # 
    if set10:
        # (V0/V-1)/(V-1/V-2)<2，倍数Mtimes
        V0_Vm1 = get_vol_growth(slide_minus1to0)
        Vm1_Vm2 = get_vol_growth(slide_minus2to1)
        vol_series = pd.concat([V0_Vm1, Vm1_Vm2],axis=1).set_axis(['V0_Vm1', 'Vm1_Vm2'], axis=1)
        times_of_change = vol_series[(vol_series['V0_Vm1']+1)/(vol_series['Vm1_Vm2']+1) < Mtimes]
        set11 = set(times_of_change.index)
        set1 = set1.intersection(set11)
        
        
    selected_codes = set1
    if selected_codes:
        return_df = get_filtered_df(new_total_df,working_days[index+20],20) # day+BDay(20)
        return_rate = get_portfolio_return(new_total_df,day,selected_codes,nday_sold)
        day_list.append(day)
        code_list.append(selected_codes)
        return_rate_total.append(return_rate)   
        company_list = today[['股票代码','股票名称','日期','成交量']].set_index('股票代码')
        company_list = company_list.loc[company_list.index.isin(selected_codes)]       
        today_pchange = pd.DataFrame(today_growth).rename(columns={'收盘涨幅对数': '今日涨幅'})
        today_pchange = today_pchange.loc[today_pchange.index.isin(selected_codes)]
        recent3daychange = pd.DataFrame(get_growth(get_filtered_df(slide_df, working_days[index], 3))).rename(columns={'收盘涨幅对数': '近三日涨幅'})
        recent3daychange = recent3daychange.loc[recent3daychange.index.isin(selected_codes)]
        D1_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+1],1))).rename(columns={'收盘涨幅对数': 'D+1涨幅'}) # day+BDay(1)
        D1_pchange = D1_pchange.loc[D1_pchange.index.isin(selected_codes)]
        D2_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+2],2))).rename(columns={'收盘涨幅对数': 'D+2涨幅'}) # day+BDay(2)
        D2_pchange = D2_pchange.loc[D2_pchange.index.isin(selected_codes)]
        D3_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+3] ,3))).rename(columns={'收盘涨幅对数': 'D+3涨幅'}) # day+BDay(3)
        D3_pchange = D3_pchange.loc[D3_pchange.index.isin(selected_codes)]
        D4_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+4],4))).rename(columns={'收盘涨幅对数': 'D+4涨幅'}) # day+BDay(4)
        D4_pchange = D4_pchange.loc[D4_pchange.index.isin(selected_codes)]
        D5_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+5] ,5))).rename(columns={'收盘涨幅对数': 'D+5涨幅'}) # day+BDay(5)
        D5_pchange = D5_pchange.loc[D5_pchange.index.isin(selected_codes)]
        D6_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+6] ,6))).rename(columns={'收盘涨幅对数': 'D+6涨幅'}) # day+BDay(6)
        D6_pchange = D6_pchange.loc[D6_pchange.index.isin(selected_codes)]
        D7_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+7] ,7))).rename(columns={'收盘涨幅对数': 'D+7涨幅'}) # day+BDay(7)
        D7_pchange = D7_pchange.loc[D7_pchange.index.isin(selected_codes)]
        D8_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+8] ,8))).rename(columns={'收盘涨幅对数': 'D+8涨幅'}) # day+BDay(8)
        D8_pchange = D8_pchange.loc[D8_pchange.index.isin(selected_codes)]
        D9_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+9] ,9))).rename(columns={'收盘涨幅对数': 'D+9涨幅'}) # day+BDay(9)
        D9_pchange = D9_pchange.loc[D9_pchange.index.isin(selected_codes)]
        D10_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+10] ,10))).rename(columns={'收盘涨幅对数': 'D+10涨幅'}) # day+BDay(10)
        D10_pchange = D10_pchange.loc[D10_pchange.index.isin(selected_codes)]
        D11_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+11] ,11))).rename(columns={'收盘涨幅对数': 'D+11涨幅'}) # day+BDay(11)
        D11_pchange = D11_pchange.loc[D11_pchange.index.isin(selected_codes)]
        D12_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+12] ,12))).rename(columns={'收盘涨幅对数': 'D+12涨幅'}) # day+BDay(12)
        D12_pchange = D12_pchange.loc[D12_pchange.index.isin(selected_codes)]
        D13_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+13] ,13))).rename(columns={'收盘涨幅对数': 'D+13涨幅'}) # day+BDay(13)
        D13_pchange = D13_pchange.loc[D13_pchange.index.isin(selected_codes)]
        D14_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+14] ,14))).rename(columns={'收盘涨幅对数': 'D+14涨幅'}) # day+BDay(14)
        D14_pchange = D14_pchange.loc[D14_pchange.index.isin(selected_codes)]
        D15_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+15] ,15))).rename(columns={'收盘涨幅对数': 'D+15涨幅'}) # day+BDay(15)
        D15_pchange = D15_pchange.loc[D15_pchange.index.isin(selected_codes)]
        D16_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+16] ,16))).rename(columns={'收盘涨幅对数': 'D+16涨幅'}) # day+BDay(16)
        D16_pchange = D16_pchange.loc[D16_pchange.index.isin(selected_codes)]
        D17_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+17] ,17))).rename(columns={'收盘涨幅对数': 'D+17涨幅'}) # day+BDay(17)
        D17_pchange = D17_pchange.loc[D17_pchange.index.isin(selected_codes)]
        D18_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+18] ,18))).rename(columns={'收盘涨幅对数': 'D+18涨幅'}) # day+BDay(18)
        D18_pchange = D18_pchange.loc[D18_pchange.index.isin(selected_codes)]
        D19_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+19] ,19))).rename(columns={'收盘涨幅对数': 'D+19涨幅'}) # day+BDay(19)
        D19_pchange = D19_pchange.loc[D19_pchange.index.isin(selected_codes)]
        D20_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+20] ,20))).rename(columns={'收盘涨幅对数': 'D+20涨幅'}) # day+BDay(20)
        D20_pchange = D20_pchange.loc[D20_pchange.index.isin(selected_codes)]
        result_series = pd.concat([company_list,today_pchange,recent3daychange,D1_pchange,D2_pchange,D3_pchange,D4_pchange,D5_pchange,D6_pchange,D7_pchange,D8_pchange,D9_pchange,D10_pchange,D11_pchange,D12_pchange,D13_pchange,D14_pchange,D15_pchange,D16_pchange,D17_pchange,D18_pchange,D19_pchange,D20_pchange],axis=1)
        
        result_df = pd.concat([result_df,result_series],axis=0)
        print('Selected codes are {}'.format(selected_codes))
        print('Getting return rate of {}'.format(return_rate))
    else:
        print('No stock is selected')
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time: ", execution_time, " seconds")
    print('\n')
    
end_time_all = time.time()
execution_time_all = end_time_all-start_time_original

print('Finish execution, using {} seconds'.format(execution_time_all))
if return_rate_total:
    average_return = sum(return_rate_total) / len(return_rate_total)
    print('Average return: {}'.format(average_return))
    portfolio_dict = {
        'Date':day_list,
        'Scode':code_list,
        'return_rate':return_rate_total
    }
    portfolio_df = pd.DataFrame(portfolio_dict)
    portfolio_df.to_excel('portfolio_return.xlsx')
    result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
    result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
    result_df['D+1涨幅'] = result_df['D+1涨幅'].astype(float)
    result_df['D+2涨幅'] = result_df['D+2涨幅'].astype(float)
    result_df['D+3涨幅'] = result_df['D+3涨幅'].astype(float)
    result_df['D+4涨幅'] = result_df['D+4涨幅'].astype(float)
    result_df['D+5涨幅'] = result_df['D+5涨幅'].astype(float)
    result_df['D+6涨幅'] = result_df['D+6涨幅'].astype(float)
    result_df['D+7涨幅'] = result_df['D+7涨幅'].astype(float)
    result_df['D+8涨幅'] = result_df['D+8涨幅'].astype(float)
    result_df['D+9涨幅'] = result_df['D+9涨幅'].astype(float)
    result_df['D+10涨幅'] = result_df['D+10涨幅'].astype(float)
    result_df['D+11涨幅'] = result_df['D+11涨幅'].astype(float)
    result_df['D+12涨幅'] = result_df['D+12涨幅'].astype(float)
    result_df['D+13涨幅'] = result_df['D+13涨幅'].astype(float)
    result_df['D+14涨幅'] = result_df['D+14涨幅'].astype(float)
    result_df['D+15涨幅'] = result_df['D+15涨幅'].astype(float)
    result_df['D+16涨幅'] = result_df['D+16涨幅'].astype(float)
    result_df['D+17涨幅'] = result_df['D+17涨幅'].astype(float)
    result_df['D+18涨幅'] = result_df['D+18涨幅'].astype(float)
    result_df['D+19涨幅'] = result_df['D+19涨幅'].astype(float)
    result_df['D+20涨幅'] = result_df['D+20涨幅'].astype(float)
    result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+1涨幅'] = result_df['D+1涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+2涨幅'] = result_df['D+2涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+3涨幅'] = result_df['D+3涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+4涨幅'] = result_df['D+4涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+5涨幅'] = result_df['D+5涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+6涨幅'] = result_df['D+6涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+7涨幅'] = result_df['D+7涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+8涨幅'] = result_df['D+8涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+9涨幅'] = result_df['D+9涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+10涨幅'] = result_df['D+10涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+11涨幅'] = result_df['D+11涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+12涨幅'] = result_df['D+12涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+13涨幅'] = result_df['D+13涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+14涨幅'] = result_df['D+14涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+15涨幅'] = result_df['D+15涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+16涨幅'] = result_df['D+16涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+17涨幅'] = result_df['D+17涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+18涨幅'] = result_df['D+18涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+19涨幅'] = result_df['D+19涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+20涨幅'] = result_df['D+20涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
    result_df.to_excel('result_df.xlsx')
else:
    print('No stock selected')

Selecting stocks at 2022-12-01 00:00:00
No stock is selected
Execution time:  0.025527238845825195  seconds


Selecting stocks at 2022-12-02 00:00:00
No stock is selected
Execution time:  0.016741514205932617  seconds


Selecting stocks at 2022-12-05 00:00:00
No stock is selected
Execution time:  0.02088761329650879  seconds


Selecting stocks at 2022-12-06 00:00:00
No stock is selected
Execution time:  0.017882823944091797  seconds


Selecting stocks at 2022-12-07 00:00:00
No stock is selected
Execution time:  0.012970447540283203  seconds


Selecting stocks at 2022-12-08 00:00:00
No stock is selected
Execution time:  0.011774063110351562  seconds


Selecting stocks at 2022-12-09 00:00:00
No stock is selected
Execution time:  0.006831645965576172  seconds


Selecting stocks at 2022-12-12 00:00:00
No stock is selected
Execution time:  0.015659332275390625  seconds


Selecting stocks at 2022-12-13 00:00:00
No stock is selected
Execution time:  0.015586137771606445  seconds


Selecting s

No stock is selected
Execution time:  2.5560245513916016  seconds


Selecting stocks at 2023-03-01 00:00:00

Selected codes are {'300496'}
Getting return rate of 0.07466451417616793
Execution time:  2.885554552078247  seconds


Selecting stocks at 2023-03-02 00:00:00

Selected codes are {'600898', '300847', '000547', '002369', '301042', '603316', '300270', '002115', '600476', '600345', '300136', '300504', '000151', '002446'}
Getting return rate of -0.014797205511352662
Execution time:  2.8993117809295654  seconds


Selecting stocks at 2023-03-03 00:00:00

Selected codes are {'002941', '301215', '300559'}
Getting return rate of -0.003962206644315791
Execution time:  2.8880813121795654  seconds


Selecting stocks at 2023-03-06 00:00:00

Selected codes are {'600764', '000852', '600737', '300140', '300456', '688120'}
Getting return rate of -0.021936117193989246
Execution time:  2.8318283557891846  seconds


Selecting stocks at 2023-03-07 00:00:00

Selected codes are {'301082', '688063', '0

Selected codes are {'688533', '002659', '839790', '001213', '002305', '600959', '000544', '600730', '002307', '301175', '002344', '601177', '603721'}
Getting return rate of -0.0010395010395010118
Execution time:  2.901413917541504  seconds


Selecting stocks at 2023-05-08 00:00:00

No stock is selected
Execution time:  2.5479393005371094  seconds


Selecting stocks at 2023-05-09 00:00:00

Selected codes are {'600797', '601059'}
Getting return rate of -0.04381976023150058
Execution time:  2.9530529975891113  seconds


Selecting stocks at 2023-05-10 00:00:00

Selected codes are {'688079'}
Getting return rate of 0.005015045135406071
Execution time:  2.957066059112549  seconds


Selecting stocks at 2023-05-11 00:00:00

No stock is selected
Execution time:  2.5735082626342773  seconds


Selecting stocks at 2023-05-12 00:00:00

Selected codes are {'300490', '688039', '301200', '600201', '601116'}
Getting return rate of -0.03145115293734213
Execution time:  2.880112409591675  seconds


Select

Selected codes are {'002941', '600425'}
Getting return rate of -0.04921165790731019
Execution time:  3.395388126373291  seconds


Selecting stocks at 2023-07-12 00:00:00

Selected codes are {'002869', '600059'}
Getting return rate of 0.0014594279042614566
Execution time:  2.9140899181365967  seconds


Selecting stocks at 2023-07-13 00:00:00

Selected codes are {'300826', '603829', '300932', '688035', '300062'}
Getting return rate of 0.006411310208778609
Execution time:  2.9404988288879395  seconds


Selecting stocks at 2023-07-14 00:00:00

Selected codes are {'301332', '300344'}
Getting return rate of -0.007029053420805953
Execution time:  2.9097652435302734  seconds


Selecting stocks at 2023-07-17 00:00:00

Selected codes are {'300541', '300306', '300290', '002876', '300608', '300249', '688132', '301337'}
Getting return rate of -0.015821342535321525
Execution time:  2.9974570274353027  seconds


Selecting stocks at 2023-07-18 00:00:00

No stock is selected
Execution time:  2.53782677

Selected codes are {'301165', '002907', '301488', '301393', '688205'}
Getting return rate of -0.03819080965960342
Execution time:  2.991572380065918  seconds


Selecting stocks at 2023-09-13 00:00:00

Selected codes are {'301165', '603177'}
Getting return rate of -0.051217198861840174
Execution time:  2.923334836959839  seconds


Selecting stocks at 2023-09-14 00:00:00

No stock is selected
Execution time:  2.589573621749878  seconds


Selecting stocks at 2023-09-15 00:00:00

Selected codes are {'300945', '600397', '000416'}
Getting return rate of 0.020143240823634834
Execution time:  2.96992564201355  seconds


Selecting stocks at 2023-09-18 00:00:00

Selected codes are {'000571', '603836', '688793'}
Getting return rate of -0.00771715721464461
Execution time:  2.9157049655914307  seconds


Selecting stocks at 2023-09-19 00:00:00

Selected codes are {'600190', '600853', '002808'}
Getting return rate of -0.023383768913342484
Execution time:  2.9477345943450928  seconds


Selecting stock

IndexError: index 239 is out of bounds for axis 0 with size 239

In [ ]:
result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
result_df['D+1涨幅'] = result_df['D+1涨幅'].astype(float)
result_df['D+2涨幅'] = result_df['D+2涨幅'].astype(float)
result_df['D+3涨幅'] = result_df['D+3涨幅'].astype(float)
result_df['D+4涨幅'] = result_df['D+4涨幅'].astype(float)
result_df['D+5涨幅'] = result_df['D+5涨幅'].astype(float)
result_df['D+6涨幅'] = result_df['D+6涨幅'].astype(float)
result_df['D+7涨幅'] = result_df['D+7涨幅'].astype(float)
result_df['D+8涨幅'] = result_df['D+8涨幅'].astype(float)
result_df['D+9涨幅'] = result_df['D+9涨幅'].astype(float)
result_df['D+10涨幅'] = result_df['D+10涨幅'].astype(float)
result_df['D+11涨幅'] = result_df['D+11涨幅'].astype(float)
result_df['D+12涨幅'] = result_df['D+12涨幅'].astype(float)
result_df['D+13涨幅'] = result_df['D+13涨幅'].astype(float)
result_df['D+14涨幅'] = result_df['D+14涨幅'].astype(float)
result_df['D+15涨幅'] = result_df['D+15涨幅'].astype(float)
result_df['D+16涨幅'] = result_df['D+16涨幅'].astype(float)
result_df['D+17涨幅'] = result_df['D+17涨幅'].astype(float)
result_df['D+18涨幅'] = result_df['D+18涨幅'].astype(float)
result_df['D+19涨幅'] = result_df['D+19涨幅'].astype(float)
result_df['D+20涨幅'] = result_df['D+20涨幅'].astype(float)
result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+1涨幅'] = result_df['D+1涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+2涨幅'] = result_df['D+2涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+3涨幅'] = result_df['D+3涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+4涨幅'] = result_df['D+4涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+5涨幅'] = result_df['D+5涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+6涨幅'] = result_df['D+6涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+7涨幅'] = result_df['D+7涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+8涨幅'] = result_df['D+8涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+9涨幅'] = result_df['D+9涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+10涨幅'] = result_df['D+10涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+11涨幅'] = result_df['D+11涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+12涨幅'] = result_df['D+12涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+13涨幅'] = result_df['D+13涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+14涨幅'] = result_df['D+14涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+15涨幅'] = result_df['D+15涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+16涨幅'] = result_df['D+16涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+17涨幅'] = result_df['D+17涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+18涨幅'] = result_df['D+18涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+19涨幅'] = result_df['D+19涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['D+20涨幅'] = result_df['D+20涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
result_df.to_excel('result_df.xlsx')